In [1]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch 
from sklearn.metrics.pairwise import cosine_similarity

C:\Users\emzou\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
import numpy as np
import ast

In [2]:
da= pd.read_csv("nov11.csv")

In [21]:
da['Tokens_E'] = da['Tokens_E'].apply(ast.literal_eval)

In [22]:
def get_dei_bigram (tokens): 
    for i in range (len(tokens) -1): 
        if tokens[i] == "dei": 
            return f"dei {tokens[i+1]}"
    return np.nan

da['Bigram']= da['Tokens_E'].apply(get_dei_bigram)

In [77]:
df = da[['Bigram', 'Tokens_E']].copy()

In [78]:
df = df.dropna()

In [80]:
df['Len'] = df['Tokens_E'].apply(len)
df['Len'].value_counts()

11     926
9      892
14     871
12     854
8      847
      ... 
106      1
181      1
221      1
143      1
177      1
Name: Len, Length: 168, dtype: int64

In [ ]:
## this needs to be fixed later:: some sentences were improperly split / need to account for weird text/punctuation. for now, going to limit data to sentences < 50 tokens.

sentlength = list(df['Len'].value_counts())
from statistics import mean
from statistics import median

mean(sentlength)
median(sentlength)

In [83]:
df = df[df['Len'] <= 50]

In [84]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')



In [89]:
def get_embedding(text): 
    inputs = tokenizer(text, return_tensors = "pt", truncation = True, padding = True)
    with torch.no_grad(): 
        outputs = model(**inputs)
    cls_embedding = outputs.last_hidden_state[:, 0, :]  
    return cls_embedding

df['Contextual_Embedding'] = df['Tokens_E'].apply(get_embedding) 

df

df['Word2'] = df['Bigram'].apply(lambda x: x.split()[1])

df['Contextual_Embedding']= df['Tokens_E'].apply(get_embedding)

df['Embedding_Dim'] = df['Contextual_Embedding'].apply(lambda x: x.size(0) if isinstance(x, torch.Tensor) else None)
inconsistent_rows = df[df['Embedding_Dim'] != 768]
print(inconsistent_rows)

dei_hire_embedding= df[df['Bigram'] == "dei hire"]['Contextual_Embedding'].values

dei_hire_tensors = [torch.tensor(emb.numpy()) if isinstance(emb, torch.Tensor) else emb for emb in dei_hire_embedding]
target_embedding = torch.stack(dei_hire_tensors).mean(dim=0)

df['Similarity'] = df['Contextual_Embedding'].apply(lambda x: cosine_similarity(x, target_embedding)[0][0]) 


C:\Users\emzou\AppData\Local\Temp\ipykernel_10908\2971193950.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Contextual_Embedding'] = df['Tokens_E'].apply(get_embedding)
C:\Users\emzou\AppData\Local\Temp\ipykernel_10908\2971193950.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Word2'] = df['Bigram'].apply(lambda x: x.split()[1])
C:\Users\emzou\AppData\Local\Temp\ipykernel_10908\2971193950.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

            Bigram                                           Tokens_E  Len  \
0        dei roles  [tumblr, girls, were, the, nerdy, outcasts, wh...   20   
1         dei good  [just, robbing, and, dispossesing, innocent, c...   31   
2         dei mind  [the, dei, mind, virus, has, infected, every, ...   11   
5        dei asset  [i, ca, tell, actual, practicable, advice, fro...   25   
6           dei is                 [dei, is, the, law, of, the, land]    7   
...            ...                                                ...  ...   
23432  dei minions  [milley, and, the, dei, minions, basically, se...   19   
23433   dei hiring  [he's, in, charge, of, dei, hiring, in, the, w...    9   
23434     dei hire                         [kamala, is, a, dei, hire]    5   
23435      dei has  [the, can, barely, gaslight, this, within, the...   35   
23436   dei troons  [dei, troons, wars, expensive, shit, and, raci...   12   

         Word2                               Contextual_Embeddi

RuntimeError: stack expects each tensor to be equal size, but got [27, 768] at entry 0 and [8, 768] at entry 1

In [ ]:

target_bigram = "dei hire"

inputs_target = tokenizer(target_bigram, return_tensors = "pt")
candidate_bigrams = list(set(df['Bigram'].dropna().unique())) 
inputs_candidates = tokenizer(candidate_bigrams, padding = True, truncation = True, return_tensors = "pt")

In [67]:


with torch.no_grad():
    target_embedding = model (**inputs_target).last_hidden_state.mean(dim = 1).squeeze().numpy()
    candidate_embeddings = model (**inputs_candidates).last_hidden_state.mean(dim = 1).numpy()

similarities = cosine_similarity([target_embedding], candidate_embeddings)

df['Similarity']= [similarities[0][i] for i in range(len(similarities[0]))]
inputs_candidates = tokenizer(list(candidate_bigrams), padding = True, truncation = True, return_tensors = "pt")

with torch.no_grad():
    target_embedding = model (**inputs_target).last_hidden_state.mean(dim = 1).squeeze().numpy()
    candidate_embeddings = model (**inputs_candidates).last_hidden_state.mean(dim = 1).numpy()

similarities = cosine_similarity([target_embedding], candidate_embeddings)

df['Similarity']= [similarities[0][i] for i in range(len(similarities[0]))]

ValueError: Length of values (3608) does not match length of index (23437)